### 1. Reading Samples from a Recorded File

In [ ]:
# reading samples from binary file

import numpy as np
from array import array
from os import stat

filename = "rx.sig"

# read in I/Q samples
n_float32 = int(stat(filename).st_size/4)
with open(filename,"rb") as binary_file:
    arr = array('f')
    arr.fromfile(binary_file,n_float32)
rx = np.array(arr.tolist()[::2])+1j*np.array(arr.tolist()[1::2])
rx /= np.amax(np.abs(rx))    # normalize magnitude
print("signal size:",rx.shape[0])


### 2. Signal Plotting

* Offline

In [ ]:
# plot small portion of signal

import matplotlib.pyplot as plt

i_start = 100000
i_end = 100500

rx_i = np.arange(i_start,i_end)    # array with indices from i_start to i_end

# plot real and imaginary parts
# ...
plt.show()

# plot magnitude and phase
# ...
plt.show()


### 3. Listening to the FM Signal

* Offline

In [ ]:
# demodulate the FM radio signal

import scipy.signal as signal
from IPython.display import Audio

rxad = np.angle(rx[1:]*np.conj(rx[:-1]))    # get frequency: derivation of phase
w = signal.decimate(signal.lfilter([0.46],[1,-0.54],rxad),25)    # lowpass filter
plt.plot(w[4000:4020])
plt.show()

Audio(data=w,rate=8000)


* Online

In [ ]:
# FM receiver, works in combination with lime_to_socket.grc

import numpy as np
import socket
import sounddevice as sd
import scipy.signal as signal
import queue

q = queue.Queue()

f_station = int(90.9e6)

sample_rate = 200000
sample_rate_audio = 8000
dec_factor = int(sample_rate/sample_rate_audio)

print("select RF frequency",f_station,"at Lime source")

data = np.array([])
def callback(outdata,frames,time,status):
    global data
    if status:
        print(status)
    while (len(data)<frames):
        x = q.get()
        data = np.append(data,x.reshape((len(x),1)))
    outdata[:] = data[:frames].reshape((frames,1))
    data = data[frames:]

with socket.socket(socket.AF_INET,socket.SOCK_STREAM) as s:
    s.bind(("",52001))
    s.listen(1)
    conn,addr = s.accept()
    with conn:
        stream = sd.OutputStream(samplerate=sample_rate_audio,channels=1,callback=callback)
        with stream:
            rx_bytes = bytearray([])
            while (True):
                # read samples
                while (len(rx_bytes)<200000):
                    rx_bytes = rx_bytes+conn.recv(16384)
                rx = np.frombuffer(rx_bytes[:200000],dtype=np.complex64)
                rxd = np.angle(rx[1:]*np.conj(rx[:-1]))
                q.put(signal.decimate(signal.lfilter([0.46],[1,-0.54],rxd),dec_factor))
                rx_bytes = rx_bytes[200000:]
conn.close()


### 4. FM Broadcast

* Offline

In [ ]:
# broadcast wav-file over FM
# works in combination with file_to_lime.grc !

import numpy as np
import soundfile as sf
import scipy.signal as signal

filename = "fm_s_out.bin"
sample_rate_out = 1e5
f_dev = 1.0    # frequency deviation

s_in,sample_rate_in = sf.read('male_speech.wav')
s_resampled = signal.resample(s_in,int(sample_rate_out/sample_rate_in*len(s_in)))
fm_s_out = ...

tx = np.zeros((2*len(fm_s_out)),dtype=np.float32)
tx[::2] = np.real(fm_s_out)
tx[1::2] = np.imag(fm_s_out)
data = tx.tobytes()
file = open(filename,"wb")
file.write(bytearray(data))

* Online

In [ ]:
# from microphone to sockets
# works in combination with socket_to_lime.grc !

import numpy as np
import scipy.signal as signal
import socket
import sounddevice as sd
import queue

sample_rate_in = 8000
sample_rate_out = 1e5
f_dev = 10.0    # frequency deviation

q = queue.Queue()

def audio_callback(indata,frames,time,status):
    q.put(indata)

with socket.socket(socket.AF_INET,socket.SOCK_STREAM) as s:
    s.bind(("",52001))
    s.listen(1)
    conn,addr = s.accept()
    with conn:
        stream = sd.InputStream(device=None,channels=1,samplerate=sample_rate_in,blocksize=1000,callback=audio_callback)
        with stream:
            while True:
                s_in = q.get()
                s_resampled = signal.resample(s_in,int(sample_rate_out/sample_rate_in*len(s_in)))
                fm_s_out = ...
                tx = np.zeros((2*len(fm_s_out)),dtype=np.float32)
                tx[::2] = np.real(fm_s_out)
                tx[1::2] = np.imag(fm_s_out)
                data = tx.tobytes()
                conn.sendall(data)

### 5. Analyzing a Signal from a Nearby Transmitter

* Offline

In [ ]:
# plot transmitted random signal

filename = "rx_random.sig"

# read in I/Q samples
n_float32 = int(stat(filename).st_size/4)
with open(filename,"rb") as binary_file:
    arr = array('f')
    arr.fromfile(binary_file,n_float32)
rx = np.array(arr.tolist()[::2])+1j*np.array(arr.tolist()[1::2])
rx /= np.amax(np.abs(rx))    # normalize magnitude
print("signal size:",rx.shape[0])

# ...
plt.show()
# ...
plt.show()